In [ ]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.websearch import SearchApiWebSearch
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.routers import TransformersZeroShotTextRouter
from haystack import Pipeline
from haystack import component, Document
from haystack.utils import Secret
from typing import List

from dotenv import load_dotenv


load_dotenv(".env")

True

In [2]:
text_router = TransformersZeroShotTextRouter(
    model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
    labels=["Politics", "Sport","Technology","Entertainment", "Business"],)
text_router.warm_up()

@component
class NewsClassifier:
    @component.output_types(documents=List[Document])
    def run(self, documents: List[Document]) -> List[Document]:
        for document in documents:
            text = document.content
            meta = document.meta
            labels = text_router.run(text)
            meta['labels'] = list(labels.keys())[0]
            
        return {"documents": documents}

Device set to use mps


### Build Haystack pipeline 

In [3]:

# Initialize pipeline
pipeline = Pipeline()
web_search = SearchApiWebSearch(top_k=5,
                                api_key=Secret.from_env_var("SEARCH_API_KEY"),
                                allowed_domains=["https://www.britannica.com/"])
link_content = LinkContentFetcher(retry_attempts=3,
                                  timeout=10)
html_to_doc = HTMLToDocument()
document_cleaner = DocumentCleaner(
                                remove_empty_lines=True,
                                remove_extra_whitespaces=True,
                                remove_repeated_substrings=False,
                                remove_substrings=['\n-']
                            )


# Add components
pipeline.add_component(name='search', instance=web_search)
pipeline.add_component(name ='fetcher' , instance= link_content)
pipeline.add_component(name='htmldocument', instance=html_to_doc)
pipeline.add_component(name='cleaner', instance=document_cleaner)
pipeline.add_component(name='classifier', instance=NewsClassifier())

# Connect components to one another
pipeline.connect("search.links", "fetcher.urls")
pipeline.connect("fetcher", "htmldocument")
pipeline.connect("htmldocument", "cleaner")
pipeline.connect("cleaner", "classifier")



🚅 Components
  - search: SearchApiWebSearch
  - fetcher: LinkContentFetcher
  - htmldocument: HTMLToDocument
  - cleaner: DocumentCleaner
  - classifier: NewsClassifier
🛤️ Connections
  - search.links -> fetcher.urls (list[str])
  - fetcher.streams -> htmldocument.sources (list[ByteStream])
  - htmldocument.documents -> cleaner.documents (list[Document])
  - cleaner.documents -> classifier.documents (list[Document])

### Use pipeline to search for all articles related to Elon Musk and extract entities

In [4]:
query = "Elon Musk"
output = pipeline.run(data={"search":{"query":query}})

In [5]:
extracted_documents = output['classifier']['documents']

In [6]:
for i in range(len(extracted_documents)):
    print(extracted_documents[i].meta)
    # to get content of the document use extracted_documents[i].content
    print("----")

{'content_type': 'text/html', 'url': 'https://www.britannica.com/money/Elon-Musk', 'labels': 'Technology'}
----
{'content_type': 'text/html', 'url': 'https://www.britannica.com/money/Zip2', 'labels': 'Technology'}
----
{'content_type': 'text/html', 'url': 'https://www.britannica.com/topic/Department-of-Government-Efficiency-United-States', 'labels': 'Politics'}
----
{'content_type': 'text/html', 'url': 'https://www.britannica.com/money/Tesla-Motors', 'labels': 'Business'}
----
{'content_type': 'text/html', 'url': 'https://www.britannica.com/money/xAI', 'labels': 'Technology'}
----
